In [41]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [42]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [43]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [44]:
def draw_styled_landmarks(image, results):
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [51]:
def view_detections():    
    cap = cv2.VideoCapture(0)
    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():

            # Read feed
            ret, frame = cap.read()

            # Make detections
            image, results = mediapipe_detection(frame, holistic)

            # Draw landmarks
            draw_styled_landmarks(image, results)

            # Show to screen
            cv2.imshow('OpenCV Feed', image)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

In [52]:
def extract_keypoints(results):
    left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([left_hand,right_hand])

In [53]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions_total = np.array(['apply-throttle', 'release-throttle', 'apply-brake','release-brake'])

# Thirty videos worth of data
sequence_limit = 30

# Videos are going to be 30 frames in length
frame_limit = 30


In [54]:
for action in actions_total: 
    for sequence in range(sequence_limit):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [57]:
cap = cv2.VideoCapture(0)

is_recording = False  # Flag to track recording state
is_done = False  # Flag to track the initial pause

# Keyboard event handler
def key_press(key):
    global is_recording,is_done
    if key == ord('s'):
        is_done = False
        is_recording = True

#copying the total actions array
actions = actions_total.copy()
actions = actions[0:1]
#Intial values of frame and video number
sequence = 0
frame_num = 0
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # Draw landmarks
        draw_styled_landmarks(image, results)
        # Display "Collecting..." message after pressing 's'
        if frame_num >=1 and is_recording:
            cv2.putText(image, "Collecting - frame {}".format(frame_num), (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4,cv2.LINE_AA)
        if is_done:
            cv2.putText(image, 'action: {} | sequence: {} '.format(actions[-1],sequence), (20,20), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.632, (0,255, 0), 4, cv2.LINE_AA)
        
        cv2.imshow('OpenCV Feed', image)
        #taking input from user
        key_selected = cv2.waitKey(1)
        # Store landmarks when 's' key is pressed
        key_press(key_selected)
        #click "s" to initiate video/frame collection
        if  is_recording and len(actions)>0:
                    keypoints = extract_keypoints(results)
                    npy_path = os.path.join(DATA_PATH, actions[-1], str(sequence), str(frame_num))
                    np.save(npy_path, keypoints)
                    print('action: {} | sequence: {} | frame: {} '.format(actions[-1],sequence,frame_num))
                    frame_num += 1
                    if frame_num == frame_limit:
                        frame_num=0
                        is_recording = False
                        is_done = True
                        sequence+=1
                    if sequence==sequence_limit:
                        sequence=0
                        actions = actions[:-1]
        elif key_selected == ord('q'):
            break
        else:
            pass
        
    cap.release()
    cv2.destroyAllWindows()

action: apply-throttle | sequence: 0 | frame: 0 
action: apply-throttle | sequence: 0 | frame: 1 
action: apply-throttle | sequence: 0 | frame: 2 
action: apply-throttle | sequence: 0 | frame: 3 
action: apply-throttle | sequence: 0 | frame: 4 
action: apply-throttle | sequence: 0 | frame: 5 
action: apply-throttle | sequence: 0 | frame: 6 
action: apply-throttle | sequence: 0 | frame: 7 
action: apply-throttle | sequence: 0 | frame: 8 
action: apply-throttle | sequence: 0 | frame: 9 
action: apply-throttle | sequence: 0 | frame: 10 
action: apply-throttle | sequence: 0 | frame: 11 
action: apply-throttle | sequence: 0 | frame: 12 
action: apply-throttle | sequence: 0 | frame: 13 
action: apply-throttle | sequence: 0 | frame: 14 
action: apply-throttle | sequence: 0 | frame: 15 
action: apply-throttle | sequence: 0 | frame: 16 
action: apply-throttle | sequence: 0 | frame: 17 
action: apply-throttle | sequence: 0 | frame: 18 
action: apply-throttle | sequence: 0 | frame: 19 
action: ap

action: apply-throttle | sequence: 5 | frame: 18 
action: apply-throttle | sequence: 5 | frame: 19 
action: apply-throttle | sequence: 5 | frame: 20 
action: apply-throttle | sequence: 5 | frame: 21 
action: apply-throttle | sequence: 5 | frame: 22 
action: apply-throttle | sequence: 5 | frame: 23 
action: apply-throttle | sequence: 5 | frame: 24 
action: apply-throttle | sequence: 5 | frame: 25 
action: apply-throttle | sequence: 5 | frame: 26 
action: apply-throttle | sequence: 5 | frame: 27 
action: apply-throttle | sequence: 5 | frame: 28 
action: apply-throttle | sequence: 5 | frame: 29 
action: apply-throttle | sequence: 6 | frame: 0 
action: apply-throttle | sequence: 6 | frame: 1 
action: apply-throttle | sequence: 6 | frame: 2 
action: apply-throttle | sequence: 6 | frame: 3 
action: apply-throttle | sequence: 6 | frame: 4 
action: apply-throttle | sequence: 6 | frame: 5 
action: apply-throttle | sequence: 6 | frame: 6 
action: apply-throttle | sequence: 6 | frame: 7 
action: 

action: apply-throttle | sequence: 11 | frame: 3 
action: apply-throttle | sequence: 11 | frame: 4 
action: apply-throttle | sequence: 11 | frame: 5 
action: apply-throttle | sequence: 11 | frame: 6 
action: apply-throttle | sequence: 11 | frame: 7 
action: apply-throttle | sequence: 11 | frame: 8 
action: apply-throttle | sequence: 11 | frame: 9 
action: apply-throttle | sequence: 11 | frame: 10 
action: apply-throttle | sequence: 11 | frame: 11 
action: apply-throttle | sequence: 11 | frame: 12 
action: apply-throttle | sequence: 11 | frame: 13 
action: apply-throttle | sequence: 11 | frame: 14 
action: apply-throttle | sequence: 11 | frame: 15 
action: apply-throttle | sequence: 11 | frame: 16 
action: apply-throttle | sequence: 11 | frame: 17 
action: apply-throttle | sequence: 11 | frame: 18 
action: apply-throttle | sequence: 11 | frame: 19 
action: apply-throttle | sequence: 11 | frame: 20 
action: apply-throttle | sequence: 11 | frame: 21 
action: apply-throttle | sequence: 11 

action: apply-throttle | sequence: 16 | frame: 15 
action: apply-throttle | sequence: 16 | frame: 16 
action: apply-throttle | sequence: 16 | frame: 17 
action: apply-throttle | sequence: 16 | frame: 18 
action: apply-throttle | sequence: 16 | frame: 19 
action: apply-throttle | sequence: 16 | frame: 20 
action: apply-throttle | sequence: 16 | frame: 21 
action: apply-throttle | sequence: 16 | frame: 22 
action: apply-throttle | sequence: 16 | frame: 23 
action: apply-throttle | sequence: 16 | frame: 24 
action: apply-throttle | sequence: 16 | frame: 25 
action: apply-throttle | sequence: 16 | frame: 26 
action: apply-throttle | sequence: 16 | frame: 27 
action: apply-throttle | sequence: 16 | frame: 28 
action: apply-throttle | sequence: 16 | frame: 29 
action: apply-throttle | sequence: 17 | frame: 0 
action: apply-throttle | sequence: 17 | frame: 1 
action: apply-throttle | sequence: 17 | frame: 2 
action: apply-throttle | sequence: 17 | frame: 3 
action: apply-throttle | sequence: 

action: apply-throttle | sequence: 21 | frame: 27 
action: apply-throttle | sequence: 21 | frame: 28 
action: apply-throttle | sequence: 21 | frame: 29 
action: apply-throttle | sequence: 22 | frame: 0 
action: apply-throttle | sequence: 22 | frame: 1 
action: apply-throttle | sequence: 22 | frame: 2 
action: apply-throttle | sequence: 22 | frame: 3 
action: apply-throttle | sequence: 22 | frame: 4 
action: apply-throttle | sequence: 22 | frame: 5 
action: apply-throttle | sequence: 22 | frame: 6 
action: apply-throttle | sequence: 22 | frame: 7 
action: apply-throttle | sequence: 22 | frame: 8 
action: apply-throttle | sequence: 22 | frame: 9 
action: apply-throttle | sequence: 22 | frame: 10 
action: apply-throttle | sequence: 22 | frame: 11 
action: apply-throttle | sequence: 22 | frame: 12 
action: apply-throttle | sequence: 22 | frame: 13 
action: apply-throttle | sequence: 22 | frame: 14 
action: apply-throttle | sequence: 22 | frame: 15 
action: apply-throttle | sequence: 22 | f

action: apply-throttle | sequence: 27 | frame: 9 
action: apply-throttle | sequence: 27 | frame: 10 
action: apply-throttle | sequence: 27 | frame: 11 
action: apply-throttle | sequence: 27 | frame: 12 
action: apply-throttle | sequence: 27 | frame: 13 
action: apply-throttle | sequence: 27 | frame: 14 
action: apply-throttle | sequence: 27 | frame: 15 
action: apply-throttle | sequence: 27 | frame: 16 
action: apply-throttle | sequence: 27 | frame: 17 
action: apply-throttle | sequence: 27 | frame: 18 
action: apply-throttle | sequence: 27 | frame: 19 
action: apply-throttle | sequence: 27 | frame: 20 
action: apply-throttle | sequence: 27 | frame: 21 
action: apply-throttle | sequence: 27 | frame: 22 
action: apply-throttle | sequence: 27 | frame: 23 
action: apply-throttle | sequence: 27 | frame: 24 
action: apply-throttle | sequence: 27 | frame: 25 
action: apply-throttle | sequence: 27 | frame: 26 
action: apply-throttle | sequence: 27 | frame: 27 
action: apply-throttle | sequenc

IndexError: index -1 is out of bounds for axis 0 with size 0